## 1. Loading and Initializing

### 1.1 Sentence processing functions

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
print("Using Tensorflow version: " + tf.__version__)
print(tf.config.list_physical_devices('GPU'))

BERT_DIR = "/home/aufish/Downloads/bert"

# try with TF2 SavedModel
# The online downloading method does not work, use pre-downloaded module
# bert_module = hub.Module("https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/1")

Using Tensorflow version: 2.1.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
bert_module = hub.KerasLayer(BERT_DIR, trainable=True)

InvalidArgumentError: assertion failed: [0] [Op:Assert] name: EagerVariableNameReuse

In [21]:
# tokenizer
from bert import tokenization

def create_tokenizer(vocab_file, do_lower_case=False):
    return tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer(BERT_DIR + "/assets/vocab.txt")

def convert_sentence_to_features(sentence, tokenizer, max_seq_len=50):
    tokens = ['[CLS]']
    tokens.extend(tokenizer.tokenize(sentence))
    if len(tokens) > max_seq_len-1:
        tokens = tokens[:max_seq_len-1]
    tokens.append('[SEP]')
    
    segment_ids = [0] * len(tokens)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_ids)

    #Zero Mask till seq_length
    zero_mask = [0] * (max_seq_len-len(tokens))
    input_ids.extend(zero_mask)
    input_mask.extend(zero_mask)
    segment_ids.extend(zero_mask)
    
    return input_ids, input_mask, segment_ids

def convert_sentences_to_features(sentences, tokenizer, max_seq_len=50):
    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []
    
    for sentence in sentences:
        input_ids, input_mask, segment_ids = convert_sentence_to_features(sentence, tokenizer, max_seq_len)
        all_input_ids.append(input_ids)
        all_input_mask.append(input_mask)
        all_segment_ids.append(segment_ids)
    
    return all_input_ids, all_input_mask, all_segment_ids

import random, copy
import numpy as np
def make_rand_mask(input_ids, input_mask, vocab_size, segment_id_vals=None):
    ''' 
    input_ids: the ids of words in the sentences
    input_mask: initial mask (1 if there is a word; 0 for padding)
    returns
    input_mask: replace one bit of 1 with 0, meaning that the word will be masked
    mask_word_ids: the id of words that are masked
    pure_ids: ids in number instead of one-hot (to generate weights per masked word)
    segment_id_vals: mark the masked word with segment id 1
    '''
    batch_size = len(input_ids)
    
    new_input_mask = copy.deepcopy(input_mask)
    mask_word_ids = np.zeros((batch_size, vocab_size))
    pure_ids = []
    segment_encodings = []
    for i in range(batch_size):
        total_word = sum(input_mask[i])
        mask_word = random.randint(0, total_word-1)
        
        pure_ids.append(input_ids[i][mask_word])
        assert new_input_mask[i][mask_word] == 1
        new_input_mask[i][mask_word] = 0
        mask_word_ids[i][input_ids[i][mask_word]] = 1.0
        
        # Make the masked word segment id 1
#         assert segment_id_vals[i][mask_word] == 0
#         segment_id_vals[i][mask_word] = 1
                
    return new_input_mask, tf.convert_to_tensor(mask_word_ids, dtype=tf.dtypes.float32), pure_ids, segment_id_vals

### 1.2 Blank filler model

In [ ]:
class WordPredictor(tf.keras.Model):
    # The output means, how possible the given word may fit into the blank
    def __init__(self, class_num, bert=bert_module, dropout=0.1):
        super(SentimentBert, self).__init__()
        self.bert = bert
        self.drop = tf.keras.layers.Dropout(rate=dropout, trainable=True)
        
        self.dense = tf.keras.layers.Dense(
            class_num,
            activation=None,
            kernel_initializer='glorot_uniform',
            name='word_prediction',
            trainable=True)
        
    def call(self, inputs):
        # When passed in, all tensors are stacked in one, split it into a list
        inputs = tf.unstack(tf.cast(inputs, tf.dtypes.int32), axis=1)
        pooled, sequential = self.bert(inputs)
        x = self.drop(sequential)
        return self.dense(x)

### 1.2.1 Sanity test

In [4]:
model = WordPredictor(1)

opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(opt, loss=tf.keras.losses.Crossentropy())

NameError: name 'WordPredictor' is not defined

## 2. Prepare data

In [8]:
# One time run, write all sentences in the json file into txt
# import json 

# DATA_FILE = "/home/aufish/Documents/ScratchGan++/scratchgan/emnlp_data/train.json"
# all_sentences = json.load(open(DATA_FILE, "r"))

# SENTENCE_FILE = "./sentences.txt"

# output_file = open(SENTENCE_FILE, "w")
# for sentence in all_sentences:
#     output_file.write(sentence['s'] + '\n')

In [22]:
import numpy as np
# extracted from emnlp
DATA_FILE = "./sentences.txt"

def parse_line(line):    
    input_ids, input_mask, segment_ids = convert_sentences_to_features([line], tokenizer, max_seq_len=30)
        
    input_mask, labels, pure_ids, segment_ids = \
        make_rand_mask(input_ids, input_mask, len(tokenizer.vocab), segment_ids)
    
    return ([input_ids, input_mask, segment_ids], labels)

def create_dataset(filename = DATA_FILE, data_size = 268528, batch_size = 10):
    dataset = tf.data.TextLineDataset([filename])
    
    dataset = dataset.map(lambda x : tf.numpy_function(parse_line, [x], [tf.int64, tf.float32]))
    
    # dataset = dataset.shuffle(data_size, reshuffle_each_iteration=True)
    
    dataset = dataset.batch(batch_size)
    
    return dataset

In [23]:
dataset = create_dataset(batch_size = 10)

In [24]:
for (bert_input, label) in dataset.take(1):
    print(bert_input)
    print(label)

tf.Tensor(
[[[[  101  1422  3509  1138  3228  1115  1177  1677  1103  1419  5302
     1185  2255  1106  1849  1157  3641  4413   117  1134  1132  6150
     2732   119   102     0     0     0     0     0]]

  [[    1     1     1     0     1     1     1     1     1     1     1
        1     1     1     1     1     1     1     1     1     1     1
        1     1     1     0     0     0     0     0]]

  [[    0     0     0     0     0     0     0     0     0     0     0
        0     0     0     0     0     0     0     0     0     0     0
        0     0     0     0     0     0     0     0]]]


 [[[  101   112  5875   146  1341  1164  2128  2673  1250   117  1133
     1272   146  1821  2041  1690  4692  1132  1541  5865   117   112
     1131  1163   119   102     0     0     0     0]]

  [[    1     1     1     1     1     1     1     1     1     1     1
        1     1     1     0     1     1     1     1     1     1     1
        1     1     1     1     0     0     0     0]]

  [[    0   